# Evaluating Johns Hopkins University's Data Vizualization of COVID-19

READ: In accordance with JHU's statement on the fitness of this data for medical purposes, I also prohibit the use of this notebook for any medical purpose. "Reliance on the Website for medical guidance or use of the Website in commerce is strictly prohibited." --https://coronavirus.jhu.edu/map.html


### Use Case

When our team decides to develop an ML product, we think about the available data we have to train the models that will make up the product and the "use case". Stakeholders who are close to our clients' needs will frame a use case like this: "_As a user, I want to_ [see a dashboard/receive an alert/run an automation] _that_ [tells me something about my system/takes some action on my system]". Then, it is our job as the AI/ML team to figure out what type of model(s) would solve that problem and whether the data provided by the client would work with such a model. 

Today, we will evaluate Johns Hopkins University's data on COVID-19 (which they've made available on GitHub: https://github.com/CSSEGISandData/COVID-19). The dashboard is super awesome! What if we want to add a comparison between the death rates? This tutorial will calculate the death rate per country given the number of daily confirmed cases and deaths, and explore options for timeseries analysis. 


## Learning Objectives: 

- Understand the concept of a "use case" or "business case" for creating a data product in a business setting. 
- Learn to see beyond the dashboard to answer more in-depth questions.
- Reinforce basic pandas library for data manipulation
- Introduction to timeseries concepts

### Read in the data. 

In [ ]:
from datetime import datetime
import pandas as pd

In [ ]:
# Some csv files from JHU's COVID-19 GitHub Repo:

jhu_csv_confirmed = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv'
jhu_csv_deaths = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv'
jhu_csv_recovered = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv'

jhu_last_daily_report = "../COVID-19/csse_covid_19_data/csse_covid_19_daily_reports/03-21-2020.csv"

In [ ]:
# read in CSVs as a dataframes
covid_confirmed_df = pd.read_csv(jhu_csv_confirmed) 
covid_deaths_df = pd.read_csv(jhu_csv_deaths) 
covid_recovered_df = pd.read_csv(jhu_csv_recovered) 

last_daily_report_df = pd.read_csv(jhu_last_daily_report)

### Check that the timeseries dataframes have the same number of geographical areas and columns 

In [ ]:
print("Number of Province/States: {}".format(len(covid_confirmed_df)))
print("Number of Countries: {}".format(len(covid_confirmed_df["Country/Region"].unique())))
print("Number of Columns: {}".format(len(covid_confirmed_df.columns)))

covid_confirmed_df.head(2)

In [ ]:
print("Number of Province/States: {}".format(len(covid_deaths_df)))
print("Number of Countries: {}".format(len(covid_deaths_df["Country/Region"].unique())))
print("Number of Columns: {}".format(len(covid_deaths_df.columns)))

covid_deaths_df.head(2)

In [ ]:
print("Number of Province/States: {}".format(len(covid_recovered_df)))
print("Number of Countries: {}".format(len(covid_recovered_df["Country/Region"].unique())))
print("Number of Columns: {}".format(len(covid_recovered_df.columns)))

covid_recovered_df.head(2)

### Check the number of geographical areas for the daily report

Note that this report consolidates the Confirmed and Recovered cases and Deaths in one dataset

In [ ]:
print("Number of Province/States: {}".format(len(last_daily_report_df)))
print("Number of Countries: {}".format(len(last_daily_report_df["Country/Region"].unique())))

last_daily_report_df.head(3)

### Data "Cleaning"

This data is already quite clean, and it is even more managable because we are working by country. We are not so much "cleaning" the data (such as correcting errors), as we are _reshaping_ it for our purposes. 

#### Confirmed Cases:

In [ ]:
# we want country data, so we won't be needing "Province/State":
confirmed_by_country_df = covid_confirmed_df.drop(['Province/State'], axis=1)
# we want all the cases of each country consolidated to one value, or the sum of the provinces, grouped by 'Country/Region':
confirmed_by_country_df = covid_confirmed_df.groupby(covid_confirmed_df['Country/Region']).sum()
# we will not need the latitude and longitude, either, as we are not going to create maps:
confirmed_by_country_df = confirmed_by_country_df.drop(['Long', 'Lat'], axis=1)

In [ ]:
# You may notice we simply re-stated the name the df as we made changes. 
# This is a quick way to change the df "in-place", but be careful to order your changes correctly! 

confirmed_by_country_df.head(2)

In [ ]:
# To prepare for a timeseries viz, a fast way to use the built-in pandas technique which uses a datetime index

# transpose the dataframe:
confirmed_by_country_datetime_df = confirmed_by_country_df.T
# Now, it LOOKS like the index is in datetime, but:
print(type(confirmed_by_country_datetime_df.index[0]))
# we need to convert the data type, which is easy with pandas' to_datetime() method:
confirmed_by_country_datetime_df.index = pd.to_datetime(confirmed_by_country_datetime_df.index)
# now we're ready to take a look.
confirmed_by_country_datetime_df.tail(2)

Note: When we transposed the dataframe, the dates that were once columns have now become the index! If you want to get a normal index at this point (and keep the Date column), you can use this code: 

In [ ]:
#confirmed_by_country_datetime_df["Date"] = confirmed_by_country_datetime_df.index
#confirmed_by_country_datetime_df.index = range(len(confirmed_by_country_datetime_df.index))

#### Deaths: 

In [ ]:
# we want country data, so we won't be needing "Province/State":
deaths_by_country_df = covid_deaths_df.drop(['Province/State'], axis=1)
# we want all the cases of each country consolidated to one value, or the sum of the provinces, grouped by 'Country/Region':
deaths_by_country_df = covid_deaths_df.groupby(covid_deaths_df['Country/Region']).sum()
# we will not need the latitude and longitude, either, as we are not going to create maps:
deaths_by_country_df = deaths_by_country_df.drop(['Long', 'Lat'], axis=1)

deaths_by_country_df.head(2)

In [ ]:
# transpose the dataframe:
deaths_by_country_datetime_df = deaths_by_country_df.T
# we need to convert the data type, which is easy with pandas' to_datetime() method:
deaths_by_country_datetime_df.index = pd.to_datetime(deaths_by_country_datetime_df.index)
# now we're ready to take a look.
deaths_by_country_datetime_df.tail(2)

#### Recovered cases:

In [ ]:
# we want country data, so we won't be needing "Province/State":
recovered_by_country_df = covid_recovered_df.drop(['Province/State'], axis=1)
# we want all the cases of each country consolidated to one value, or the sum of the provinces, grouped by 'Country/Region':
recovered_by_country_df = covid_recovered_df.groupby(covid_recovered_df['Country/Region']).sum()
# we will not need the latitude and longitude, either, as we are not going to create maps:
recovered_by_country_df = recovered_by_country_df.drop(['Long', 'Lat'], axis=1)

recovered_by_country_df.head(2)

In [ ]:
# transpose the dataframe:
recovered_by_country_datetime_df = recovered_by_country_df.T
# we need to convert the data type, which is easy with pandas' to_datetime() method:
recovered_by_country_datetime_df.index = pd.to_datetime(recovered_by_country_datetime_df.index)
# now we're ready to take a look.
recovered_by_country_datetime_df.tail(2)

#### Last Daily Report:

In [ ]:
# we want country data, so we won't be needing "Province/State":
last_daily_report_df = last_daily_report_df.drop(['Province/State'], axis=1)
# we want all the cases of each country consolidated to one value, or the sum of the provinces, grouped by 'Country/Region':
last_daily_report_df = last_daily_report_df.groupby(last_daily_report_df['Country/Region']).sum()
# we will not need the latitude and longitude, either, as we are not going to create maps:
last_daily_report_df = last_daily_report_df.drop(['Longitude', 'Latitude'], axis=1)

last_daily_report_df.head(2)

## Calculating the death rate

One very interesting problem I have seen is the difficulty in understanding the apparent differences in death rates in different geographical areas. There are a LOT of theories out there, but as data scientists and data analysts, we just want to look at what the data tells us. 

In [ ]:
# we can perform operations on the columns to calculate the rates: 
last_daily_report_df['Death_Rate'] = last_daily_report_df['Deaths']/last_daily_report_df['Confirmed']
last_daily_report_df['Recovery_Rate'] = last_daily_report_df['Recovered']/last_daily_report_df['Confirmed']

last_daily_report_df.head(2)

## Data Viz Fun Time! 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as pyplot

In [ ]:
death_recovery_country_comparison_df = last_daily_report_df.T

death_recovery_country_comparison_df[['New Zealand', 'US', 'China', 'Italy', 'Spain']].plot.bar()
# China skews the figure a bit
death_recovery_country_comparison_df[['New Zealand', 'US', 'Italy', 'Spain']].plot.bar()

In [ ]:
rate_comparisons_by_country_df = last_daily_report_df[["Death_Rate", "Recovery_Rate"]].T

rate_comparisons_by_country_df[['Korea, South', 'US', 'Italy', 'Spain']].plot.bar()

#### Time series visualizations

In [ ]:
# we have a lot of countries, so if we throw them all at the visualization, it will look messy: 

confirmed_by_country_datetime_df.resample('D').sum().plot()

# EWWW! 

#### Let's take a look at the most severely hit countries and the least severely hit

Remember, the timeseries dataframes are cumulative counts, so the last date (row) in the dataframe is all we need to find our target countries. Also (BONUS!) we get to learn how to use a datetime dataframe index! 

Another way is to take the Confirmed column from the last_daily_report, but we're going to pretend we only have the timeseries.  


In [ ]:
# index the last date:
last_row = confirmed_by_country_datetime_df.loc[datetime(2020, 3, 21)]
# now sort it using pandas' sort_values(). The axis defaults to row, but I add it, anyway:
sorted_last_row = last_row.sort_values(axis=0)
# now, we have a good sample of the least and hardest hit countries. 
sorted_last_row


We may want to select some countries with more than one case, just to make the data viz more interesting, with the caveat that (of course) we are showing _"lesser"_ impacted countries, not _"least"_. 

In [ ]:
# let's check out the middle a bit to see if there's any good stuff in there:
sorted_last_row[45:100]


In [ ]:
# I'm interested in the 10 countries up to and including New Zealand. Let's check out New Zealand specifically:
print("New Zealand confirmed cases: {}".format(sorted_last_row["New Zealand"]))
# but how do I get the rest of them? The index of this series is made of strings!

# make country a column in a df, and add a numerical index:
last_row_df = sorted_last_row.to_frame()
last_row_df["Country"] = sorted_last_row.index
last_row_df.index = range(len(last_row_df.index))
last_row_df.head(2)

In [ ]:
# find New Zealand again:
last_row_df[last_row_df["Country"] == "New Zealand"]

In [ ]:
# and subtract from the index number and then slice it again:
lesser_impacted = last_row_df[66:75]
lesser_impacted

In [ ]:
# now, let's just slice off the top:
most_impacted = last_row_df[-9:]
most_impacted

Now we can use these Country columns as lists to query the larger timeseries dataset:

In [ ]:
lesser_impacted_list = [country for country in lesser_impacted["Country"]]
most_impacted_list = [country for country in most_impacted["Country"]]
lesser_impacted_list

In [ ]:
lesser_impacted_confirmed_df = confirmed_by_country_datetime_df[lesser_impacted_list]
lesser_impacted_deaths_df = deaths_by_country_datetime_df[lesser_impacted_list]
lesser_impacted_deaths_df.head(2)

In [ ]:
most_impacted_confirmed_df = confirmed_by_country_datetime_df[most_impacted_list]
most_impacted_deaths_df = deaths_by_country_datetime_df[most_impacted_list]
most_impacted_confirmed_df.head(2)

#### Now, let's try those visualizations again: 

In [ ]:
lesser_impacted_confirmed_df.resample('D').sum().plot()

In [ ]:
most_impacted_confirmed_df.resample('D').sum().plot()

#### Important: Remember that your stakeholders have been looking forward to your analysis/results all day (or more!). The vizualizations are the part they've been waiting for most of all! So, go on, grab a few extra bells and whistles to make it more appealing. 

In [ ]:
import matplotlib.pyplot as plt
plt.style.use(['dark_background'])
import matplotlib
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12

In [ ]:
lesser_impacted_deaths_df.plot(figsize=(15, 6))
plt.show()

In [ ]:
plt.style.use(['ggplot'])

most_impacted_deaths_df.plot(figsize=(15, 6))
plt.show()

# Further Study and More Questions: 

- Does the recovery rate contribute to the outcomes for the country? 
- What is the shape of the death rate over time?
- Can we compare daily changes?
- What kinds of models can we build on top of these findings? 

# Sources: 

- https://coronavirus.jhu.edu/map.html
- https://www.jstor.org/stable/2352662?read-now=1&seq=16#page_scan_tab_contents
- https://github.com/CSSEGISandData/COVID-19
- https://hbr.org/1971/07/how-to-choose-the-right-forecasting-technique
- https://pandas.pydata.org/pandas-docs/stable/index.html
- https://matplotlib.org/tutorials/introductory/customizing.html#sphx-glr-tutorials-introductory-customizing-py

# CODE COMODE

Data Science in the field often requires finding out what useful questions can be answered with a dataset _as it is_ rather than a dataset we as professionals would like to have. For instance, I'd like to re-create the "Titanic problem" from Kaggle by calculating the probabilty of death given their country's case history and number of tests currently available. (If you haven't done so yet, familiarize yourself with Kaggle datasets. There are even great datasets on COVID-19, including this one from from South Korea: https://www.kaggle.com/kimjihoo/coronavirusdataset!) The data I was able to find for this tutorial, however, wouldn't be sufficient to build such a model.  

In [ ]:
# If I want it indexed by datetime:

# confirmed_by_country_datetime_df = confirmed_by_country_df.T
# confirmed_by_country_datetime_df.columns
# print(type(confirmed_by_country_datetime_df.index[0]))

# confirmed_by_country_datetime_df[datetime(2020, 3, 20):]

# clipped_series = confirmed_by_country_datetime_df['Afghanistan'][datetime(2020, 3, 20):]
# clipped_series.index

# confirmed_by_country_datetime_df.resample('D').mean().plot()

In [ ]:
# If I want to predict the death rate by the incubation period:

# def country_first_2_weeks(df, country):
#     first_2_weeks_list = []
#     for value in df[country]:
#         if value > 0:
#             first_2_weeks_list.append(value)
#     return first_2_weeks_list[:14]

# country_first_2_weeks(confirmed_by_country_datetime_df, "China")

# def country_first_2_weeks(df):
# first_2_weeks_list = []

# for i, j in confirmed_by_country_df[:10].iterrows():
#     print (i, j[0:3][0])
    
#     if value > 0:
#         print(value)

# for i, j in confirmed_by_country_datetime_df[50:].iterrows(): 
#     print(i ,j[0:1]) 
# country_from_first_confirmed_dict = {}
